In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import mvapi as mv

import importlib
def reload():
#     global Product
    importlib.reload(mv)
#     Product = mv.A/ccumulator(api_token ='')
reload()

In [75]:
def convert_np_types(obj):
    if isinstance(obj, (np.integer, np.int_)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float_)):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy arrays to lists
    else:
        raise TypeError(f"Type {type(obj)} not serializable")

In [84]:
import tkinter as tk
from tkinter import ttk
import mvapi as mv
import json
import inspect
import importlib
importlib.reload(mv) 
import pandas as pd
from tabulate import tabulate

class MvapiGUI:
    def __init__(self, master):
        self.master = master
        master.title("MVAPI GUI")
        master.geometry("1600x1000")

        self.Product = mv.Accumulator()
        self.api_token = 0

        self.create_frames()
        self.create_api_frame()
        self.create_endpoint_frame()
        self.create_button_frame()
        self.create_product_dropdown()
        self.create_check_dropdown()
        self.create_param_frames()
        self.create_output_frame()
        self.create_export_subframe()
        self.create_status_bar()

    def create_frames(self):
        # Create a canvas and scrollbar for the left frame with defined width
        left_frame_canvas = tk.Canvas(self.master, width=300)  # Set a fixed width for left frame canvas
        # Create the right frame as usual, ensuring it fills the remaining space
        self.right_frame = tk.Frame(self.master)
        self.right_frame.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH, padx=(0, 5), pady=5)

        # Scrollbar for left frame canvas
        left_scrollbar = tk.Scrollbar(self.master, orient="vertical", command=left_frame_canvas.yview)
        left_frame_canvas.configure(yscrollcommand=left_scrollbar.set)

        # Pack the scrollbar and left frame canvas side by side
        left_scrollbar.pack(side=tk.LEFT, fill=tk.Y)
        left_frame_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)  # Set expand to False

        # Create an internal frame for widgets within the left frame canvas
        self.left_frame = tk.Frame(left_frame_canvas)
        self.left_frame.bind(
            "<Configure>",
            lambda e: left_frame_canvas.configure(scrollregion=left_frame_canvas.bbox("all"))
        )

        # Add the frame to the canvas
        left_frame_canvas.create_window((0, 0), window=self.left_frame, anchor="nw")

        # Smooth scrolling setup
        def on_mouse_wheel(event):
            if event.num == 5 or event.delta < 0:
                left_frame_canvas.yview_scroll(1, "units")  # Scroll down
            elif event.num == 4 or event.delta > 0:
                left_frame_canvas.yview_scroll(-1, "units")  # Scroll up

        self.left_frame.bind_all("<MouseWheel>", on_mouse_wheel)   # For Windows
        self.left_frame.bind_all("<Button-4>", on_mouse_wheel)     # For Linux, scroll up
        self.left_frame.bind_all("<Button-5>", on_mouse_wheel)     # For Linux, scroll down

    def create_api_frame(self):
        api_frame = tk.Frame(self.left_frame, height=100)
        api_frame.pack(fill=tk.X, padx=10, pady=10)

        tk.Label(api_frame, text="API Token:").pack(anchor=tk.W)
        self.api_entry = tk.Entry(api_frame, width=30)
        self.api_entry.pack(anchor=tk.W)

        button_container = tk.Frame(api_frame)
        button_container.pack(anchor=tk.W, pady=5)

        tk.Button(button_container, text="Save", command=self.set_api_token).pack(side=tk.LEFT, padx=5)
        tk.Button(button_container, text="Delete", command=self.delete_api_token).pack(side=tk.LEFT)

    def create_endpoint_frame(self):
        endpoint_frame = tk.Frame(self.left_frame, height=100)
        endpoint_frame.pack(fill=tk.X, padx=10, pady=10)

        tk.Label(endpoint_frame, text="Endpoint:").pack(anchor=tk.W)
        self.endpoint_entry = tk.Entry(endpoint_frame, width=30)
        self.endpoint_entry.pack(anchor=tk.W)

        button_container = tk.Frame(endpoint_frame)
        button_container.pack(anchor=tk.W, pady=5)

        tk.Button(button_container, text="Save", command=self.set_endpoint).pack(side=tk.LEFT, padx=5)
#         tk.Button(button_container, text="Delete", command=self.delete_api_token).pack(side=tk.LEFT)
        
    def create_button_frame(self):
        button_frame = tk.Frame(self.left_frame)
        button_frame.pack(fill=tk.X, padx=10, pady=(2,2))

        tk.Button(button_frame, text="Buy Contract", command=self.buy_contract).pack(fill=tk.X, pady=5)
        tk.Button(button_frame, text="Price Proposal", command=self.price_proposal).pack(fill=tk.X, pady=5)
        tk.Button(button_frame, text="Sell Contract", command=self.sell_contract).pack(fill=tk.X, pady=5)
        tk.Button(button_frame, text="Sell All Contracts", command=self.sell_all).pack(fill=tk.X, pady=5)

    def create_product_dropdown(self):
        product_types = [name for name, obj in inspect.getmembers(mv) if inspect.isclass(obj) and name != 'Product']
        tk.Label(self.left_frame, text="Available Product Types").pack()
        self.product_box = ttk.Combobox(self.left_frame, values=product_types)
        self.product_box.pack(fill=tk.X, padx=10)
        self.product_box.current(0)
        self.product_box.bind('<<ComboboxSelected>>', self.change_product)

    def create_check_dropdown(self):
        tk.Label(self.left_frame, text="Available Check Functions").pack()
        self.check_box = ttk.Combobox(self.left_frame, values=self.get_check_options())
        self.check_box.pack(fill=tk.X, padx=10)
        self.check_box.current(0)
#         self.check_box.bind('<<ComboboxSelected>>', self.update_check_options)

    def create_param_frames(self):
        self.check_param_frame = tk.Frame(self.left_frame, bd=2, relief=tk.SOLID)
        self.check_param_frame.pack(fill=tk.BOTH, pady=(10,0))

        self.param_frame = tk.Frame(self.left_frame, bd=2, relief=tk.SOLID)
        self.param_frame.pack(expand=True, fill=tk.BOTH, pady=(10,0))

        self.update_check_params(self.get_check_input_params())
        self.update_user_input_params(self.get_user_input_params())

    def create_output_frame(self):
        output_frame = tk.Frame(self.right_frame, bd=2, relief=tk.SOLID)
        output_frame.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        tk.Label(output_frame, text="Output").pack(anchor=tk.N, padx=10, pady=5)
        self.output_text = tk.Text(output_frame, font=("Menlo", 14))
        self.output_text.pack(expand=True, fill=tk.BOTH)
        
    def create_export_subframe(self):
        # Create a subframe within the output frame
        export_frame = tk.Frame(self.right_frame, bd=2, relief=tk.SOLID)
        export_frame.pack(side=tk.BOTTOM, fill=tk.X)#, expand=True)

        # Input for file name
        tk.Label(export_frame, text="File Name:").pack(side=tk.LEFT, padx=5)
        self.filename_entry = tk.Entry(export_frame, width=20)
        self.filename_entry.pack(side=tk.LEFT, padx=5)

        # Dropdown for timestamp option
        tk.Label(export_frame, text="Save Timestamp:").pack(side=tk.LEFT, padx=5)
        self.timestamp_var = tk.StringVar()
        timestamp_dropdown = ttk.Combobox(export_frame, textvariable=self.timestamp_var, values=["Yes", "No"], width=10)
        timestamp_dropdown.pack(side=tk.LEFT, padx=5)
        timestamp_dropdown.current(1)  # Default to "No"

        # Button for exporting data
        export_button = tk.Button(export_frame, text="Export Data", command=self.export_data_action)
        export_button.pack(side=tk.LEFT, padx=10)
        
    def create_status_bar(self):
        status_frame = tk.Frame(self.left_frame, height=20)
        status_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=5, pady=(0,5))
        tk.Label(status_frame, text="Status: ").pack(side=tk.LEFT, anchor=tk.W, pady=2)
        self.status_output = tk.Label(status_frame, text="API Token not set.", anchor='w')
        self.status_output.pack(side=tk.LEFT, fill=tk.X)

    def set_api_token(self):
        self.api_token = self.api_entry.get()
        self.Product.api_token = self.api_token
        self.update_status(f"API Token set to: {self.api_token}.")
        self.update_output(json.dumps(self.Product.last_msg, indent=1))

    def set_endpoint(self):
        self.endpoint = self.endpoint_entry.get()
        self.Product.endpoint = self.endpoint
        self.update_status(f"Endpoint changed.")
        self.update_output(json.dumps(self.Product.last_msg, indent=1))
        
    def delete_api_token(self):
        self.api_token = 0
        self.Product.api_token = self.api_token
        self.Product.create_ws()
        self.update_status("API Token deleted.")
        self.update_output("API token deleted.\n")

    def update_status(self, status):
        self.status_output.config(text=status)
        self.master.update()

    def update_output(self, string, clear=True):
        if clear:
            self.output_text.delete("1.0", tk.END)
        self.output_text.insert(tk.END, string)
        self.master.update()

    def buy_contract(self, **kwargs):
        self.update_status("Attempting to buy contract.")
        temp_flag = self.Product.buy_contract(**kwargs)
        self.update_output(json.dumps(self.Product.last_msg, indent=1))
        self.update_status("Bought contract." if temp_flag else "Failed to buy.")
        
    def price_proposal(self, **kwargs):
        self.update_status("Attempting to get price proposal.")
        temp_flag = self.Product.price_proposal(**kwargs)
        self.update_output(json.dumps(self.Product.last_msg, indent=1))
        self.update_status("Got price proposal." if temp_flag else "Failed to get price proposal.")
        
    def sell_contract(self, **kwargs):
        self.update_status("Attempting to sell contract.")
        temp_flag = self.Product.sell_contract(**kwargs)
        self.update_output(json.dumps(self.Product.last_msg, indent=1))
        self.update_status("Sold contract." if temp_flag else "Failed to sell.")

    def sell_all(self, **kwargs):
        temp_flag = self.Product.sell_all_contracts(**kwargs)
        self.output_text.insert(tk.END, json.dumps(self.Product.last_msg, indent=1))

    def change_product(self, event=None):
        product_type = self.product_box.get()
        if product_type != self.Product.species:
            self.update_status(('Changing', self.Product.species, 'to', product_type))
            get_class = getattr(mv, product_type)
            self.Product = get_class(**self.Product.__dict__)
            self.update_check_options()
            self.update_status(f"Product type changed to {product_type}.")
            self.update_output(f"Current product type is {self.Product.species}.")

    def get_check_options(self):
        return [s for s in dir(self.Product) if s.startswith('check_')]

    def update_check_options(self, event=None):
        self.check_box['values'] = self.get_check_options()
        self.check_box.current(0)
        self.update_check_params(self.get_check_input_params())
        self.update_user_input_params(self.get_user_input_params())

    def get_check_input_params(self):
        check_input_params = list(inspect.signature(getattr(self.Product, self.check_box.get())).parameters.keys())
        if 'kwargs' in check_input_params:
            check_input_params.remove('kwargs')
        return check_input_params

    def get_user_input_params(self):
        return self.Product.user_input_params

    def update_check_params(self, new_options):
        for widget in self.check_param_frame.winfo_children():
            widget.destroy()
        for opt in new_options:
            tk.Label(self.check_param_frame, text=f"{opt}:").pack(anchor=tk.W, padx=10, pady=(5, 0))
            tk.Entry(self.check_param_frame).pack(fill=tk.X, padx=10, pady=5)
        tk.Button(self.check_param_frame, text="Run Check", command=self.run_check).pack(pady=(5,5))

    def update_user_input_params(self, new_options):
        for widget in self.param_frame.winfo_children():
            widget.destroy()
        for opt in new_options:
            tk.Label(self.param_frame, text=f"{opt}:").pack(anchor=tk.W, padx=10, pady=(5, 0))
            tk.Entry(self.param_frame).pack(fill=tk.X, padx=10, pady=5)
        tk.Button(self.param_frame, text="Update Parameters", command=self.set_user_input_params).pack(pady=(5,5))

    def process_params(self, params_dict):
        temp_dict = {}
        for key, value in params_dict.items():
            if value != '':
                if key in ['runs', 'duration', 'show']:
                    temp_dict[key] = int(value)
                elif key in ['stake', 'take_profit', 'stop_loss', 'strike', 'growth_rate']:
                    temp_dict[key] = float(value)
                else:
                    temp_dict[key] = value
        return temp_dict

    def set_check_params(self):
        params_dict = {}
        for widget in self.check_param_frame.winfo_children():
            if isinstance(widget, tk.Label):
                option_name = widget.cget("text").strip(":")
            elif isinstance(widget, tk.Entry):
                option_value = widget.get()
                params_dict[option_name] = option_value
        return self.process_params(params_dict)

    def set_user_input_params(self):
        params_dict = {}
        for widget in self.param_frame.winfo_children():
            if isinstance(widget, tk.Label):
                option_name = widget.cget("text").strip(":")
            elif isinstance(widget, tk.Entry):
                option_value = widget.get()
                params_dict[option_name] = option_value
        params_dict = self.process_params(params_dict)
        self.Product._set_attributes(**params_dict)
        self.update_output('Contract Parameters set to:\n')
        self.update_output(str(params_dict), clear=0)

    def run_check(self):
        check_type = self.check_box.get()
        self.update_status(('Running', check_type))
        index = len(self.Product.GUI_data)
        results_df = getattr(self.Product, check_type)(**self.set_check_params())
        self.update_output(results_df.to_markdown())
        self.update_output('\n' + json.dumps(self.Product.GUI_data[index:], indent=1, default=convert_np_types), clear=False)
        self.update_status((check_type, 'complete'))
        

    def export_data_action(self):
        # Get values from input fields
        filename = self.filename_entry.get()
        timestamp = self.timestamp_var.get()
        # Call export_data function from the Product class
        self.update_status(('Exporting data'))
        file_path = self.Product.export_data(file_name=filename, save_timestamp=(True if timestamp == "Yes" else False))
        self.update_output('Data exported as ' + file_path)     
        self.update_status(('Data exported as ' + file_path))
        
root = tk.Tk()
gui = MvapiGUI(root)
root.mainloop()

Stats for Payout check


,cases,pass,fail
0,2,2,0


Stats for payout_check check


,Category,cases,pass,fail
0,total,0,0,0
1,ko,0,0,0
2,sold,0,0,0
3,bid,0,0,0


Stats for barrier_stats check


,cases,pass,fail
0,7,7,0


Stats for barrier_check check


,cases,pass,fail
0,2,2,0


In [81]:
reload()

In [80]:
type(gui.Product.check_barrier())

Stats for Barrier check


,cases,pass,fail
0,2,2,0


NoneType